# PFG 5005 - Estudo Dirigido 1: Seções de Poincaré para a Hamiltoniana de Hénon-Heiles

Neste notebook, implementaremos um método de integração numérica simplético para a Hamiltoniana de Hénon-Heiles e o algoritmo de Hénon para construir seções de Poincaré.

### 5.1. Equações do Método de Euler Simplético

A Hamiltoniana de Hénon-Heiles é dada por:

$$H=\frac{1}{2}(p_{1}^{2}+p_{2}^{2}+q_{1}^{2}+q_{2}^{2})+q_{1}^{2}q_{2}-\frac{1}{3}q_{2}^{3}.$$

Para aplicar o método de Euler simplético, precisamos das equações de movimento, que são as derivadas da Hamiltoniana em relação às posições e momentos.

$$\dot{q_1} = \frac{\partial H}{\partial p_1} = p_1$$
$$\dot{q_2} = \frac{\partial H}{\partial p_2} = p_2$$
$$\dot{p_1} = -\frac{\partial H}{\partial q_1} = -(q_1 + 2q_1q_2)$$
$$\dot{p_2} = -\frac{\partial H}{\partial q_2} = -(q_2 + q_1^2 - q_2^2)$$


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

def H(q1, q2, p1, p2):
    """
    Função da Hamiltoniana de Hénon-Heiles.
    """
    return 0.5 * (p1**2 + p2**2 + q1**2 + q2**2) + q1**2 * q2 - (1/3) * q2**3

### 5.2. Implementação da construção da seção de Poincaré

Esta célula contém a lógica principal para a simulação. A função `get_poincare_points` implementa o integrador de Euler Simplético e o algoritmo de Hénon para detectar e registrar os pontos da seção de Poincaré.

A seção de Poincaré será construída no plano $(q_2, p_2)$ para a superfície de seção $q_1=0$, com a condição adicional de $p_1 \ge 0$.

O algoritmo de Hénon envolve um segundo passo de integração para encontrar o ponto exato de interseção com a superfície de seção. 
Para isso, reformulamos as equações de movimento, usando $q_1$ como a variável independente:

$$\frac{dq_2}{dq_1} = \frac{\dot{q_2}}{\dot{q_1}} = \frac{p_2}{p_1}$$
$$\frac{dp_2}{dq_1} = \frac{\dot{p_2}}{\dot{q_1}} = \frac{-(q_2 + q_1^2 - q_2^2)}{p_1}$$

O método de Euler Simplético para esta Hamiltoniana separável é um mapa que atualiza as posições e momentos em duas etapas:

1.  **Atualização das Posições:**
    $$q_1^{(n+1)}=q_1^{(n)}+\Delta t p_1^{(n)}$$
    $$q_2^{(n+1)}=q_2^{(n)}+\Delta t p_2^{(n)}$$

2.  **Atualização dos Momentos (usando as novas posições):**
    $$p_1^{(n+1)}=p_1^{(n)}-\Delta t (q_1^{(n+1)} + 2q_1^{(n+1)}q_2^{(n+1)})$$
    $$p_2^{(n+1)}=p_2^{(n)}-\Delta t (q_2^{(n+1)} + (q_1^{(n+1)})^2 - (q_2^{(n+1)})^2)$$

In [3]:
def euler_symplectic_step(q1, q2, p1, p2, dt):
    """
    Realiza um passo de integração usando o método de Euler Simplético.
    """
    # Atualiza as posições
    q1_new = q1 + dt * p1
    q2_new = q2 + dt * p2
    
    # Atualiza os momentos usando as novas posições
    p1_new = p1 - dt * (q1_new + 2 * q1_new * q2_new)
    p2_new = p2 - dt * (q2_new + q1_new**2 - q2_new**2)
    
    return q1_new, q2_new, p1_new, p2_new

In [4]:
def get_poincare_points(initial_state, E_target, dt, num_steps):
    # (código da sua função)
    q1, q2, p1, p2 = initial_state

    poincare_points_q2 = []
    poincare_points_p2 = []
    
    # Loop de integração
    for n in range(num_steps):
        # Armazena o estado anterior para detecção de cruzeiro
        q1_old, q2_old, p1_old, p2_old = q1, q2, p1, p2
        
        # Passo de integração principal
        q1, q2, p1, p2 = euler_symplectic_step(q1_old, q2_old, p1_old, p2_old, dt)
        
        # Verifica a condição de cruzamento (passagem de q1=0)
        # e a condição p1 >= 0
        if q1_old * q1 < 0 and p1 >= 0:
            # Algoritmo de Hénon para encontrar o ponto exato
            d_q1 = 0 - q1_old
            
            # Use um passo de Euler para encontrar o ponto de interseção
            q2_intersec = q2_old + d_q1 * (p2_old / p1_old)
            p2_intersec = p2_old + d_q1 * (-(q2_old + q1_old**2 - q2_old**2) / p1_old)
            
            poincare_points_q2.append(q2_intersec)
            poincare_points_p2.append(p2_intersec)
            
    return poincare_points_q2, poincare_points_p2

### 5.3. Seção de Poincaré para E variando de 0.0050 a 0.1667 via nós HPC distribuídos


Vamos fazer a mesma coisa para cada energia na lista: 0.0050, 0.0070, 0.0089, 0.0100, 0.0117, 0.0133, 0.0148, 0.0167, 0.0183, 0.0188, 0.0217, 0.0250, 0.0267, 0.0300, 0.0333, 0.0350, 0.0383, 0.0417, 0.0433, 0.0467, 0.0500, 0.0517, 0.0550, 0.0583, 0.0600, 0.0633, 0.0667, 0.0683, 0.0717, 0.0750, 0.0767, 0.0800, 0.0833, 0.0850, 0.0883, 0.0917, 0.0933, 0.0967, 0.1000, 0.1017, 0.1050, 0.1083, 0.1100, 0.1133, 0.1167, 0.1183, 0.1217, 0.1250, 0.1267, 0.1300, 0.1333, 0.1350, 0.1383, 0.1417, 0.1433, 0.1467, 0.1500, 0.1517, 0.1550, 0.1583, 0.1600, 0.1633, 0.1667 , variando também as trajetórias em cada energia para 50 valores de q e 50 valores de p gerando 2500 condições iniciais distintas.




In [4]:
import os, numpy as np, matplotlib.pyplot as plt
from dask.distributed import Client, wait
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    queue="normal",               # ajuste p/ sua fila
    cores=1,                      # por job
    memory="8GB",                # por job
    walltime="02:00:00",
    job_extra=["--exclusive"],    # opcional
    env_extra=[
        "module load python/3.11",              # ajuste para seu módulo
        "source ~/venvs/py311/bin/activate"     # ou conda activate ...
    ],
)

# quantos jobs simultâneos? (cada job = 8 cores, 32GB no exemplo)
cluster.scale(jobs=30)
client = Client(cluster)
client

/Users/nara/Repository/Attention/Academic-Codex/PGF5005-Mecaninca-Classica/.venv/lib/python3.13/site-packages/dask_jobqueue/core.py:266: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/Users/nara/Repository/Attention/Academic-Codex/PGF5005-Mecaninca-Classica/.venv/lib/python3.13/site-packages/dask_jobqueue/core.py:285: FutureWarning: env_extra has been renamed to job_script_prologue. You are still using it (even if only set to []; please also check config files). If you did not set job_script_prologue yet, env_extra will be respected for now, but it will be removed in a future release. If you already set job_script_prologue, env_extra is ignor

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.5.2.120:62866/status,
Dashboard: http://10.5.2.120:62866/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.5.2.120:62867,Workers: 0
Dashboard: http://10.5.2.120:62866/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [22]:
E_list = [
    0.0050, 0.0070, 0.0089, 0.0100, 0.0117, 0.0133, 0.0148, 0.0167, 0.0183, 
    0.0188, 0.0217, 0.0250, 0.0267, 0.0300, 0.0333, 0.0350, 0.0383, 0.0417, 
    0.0433, 0.0467, 0.0500, 0.0517, 0.0550, 0.0583, 0.0600, 0.0633, 0.0667, 
    0.0683, 0.0717, 0.0750, 0.0767, 0.0800, 0.0833, 0.0850, 0.0883, 0.0917, 
    0.0933, 0.0967, 0.1000, 0.1017, 0.1050, 0.1083, 0.1100, 0.1133, 0.1167, 
    0.1183, 0.1217, 0.1250, 0.1267, 0.1300, 0.1333, 0.1350, 0.1383, 0.1417, 
    0.1433, 0.1467, 0.1500, 0.1517, 0.1550, 0.1583, 0.1600, 0.1633, 0.1667
]

In [28]:
# Parâmetros da simulação
dt = 0.01
num_steps = 10**6

# Geração de múltiplas condições iniciais em uma grade
# q2_vals = np.linspace(-0.5, 0.5, 50)
# p2_vals = np.linspace(-0.5, 0.5, 50)
# total_trajectories = len(q2_vals) * len(p2_vals)

# Geração de múltiplas condições iniciais em uma grade
q2_vals_grid = np.linspace(-0.5, 0.5, 50)
p2_vals_grid = np.linspace(-0.5, 0.5, 50)
q2_p2_combinations = np.array(np.meshgrid(q2_vals_grid, p2_vals_grid)).T.reshape(-1, 2)
total_combinations = len(q2_p2_combinations)

# Define o número inicial e final de trajetórias
min_trajectories = 350
max_trajectories = 1000

In [30]:
# Cria a pasta 'imagens' para guardar os gráficos de cada seção de Poincaré
output_dir = "imagens"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Pasta '{output_dir}' criada.")

# Define limites fixos para os eixos X e Y para todos os gráficos
# Isso garante que a escala não mude, o que é ideal para um GIF.
q2_min, q2_max = -0.6, 0.6
p2_min, p2_max = -0.6, 0.6

Pasta 'imagens' criada.


In [ ]:
# A lista de combinações selecionadas, que irá aumentar conforme aumenta a energia
selected_combinations = np.empty((0, 2))
# O conjunto de índices já presentes na simulação
simulated_indices = set()

# Itera sobre a lista de energias com um índice
for i, E_target in enumerate(E_list):

    all_q2_points = []
    all_p2_points = []
    num_valid_initials = 0

    # Calcula o número total de trajetórias que deveriam ser testadas para essa energia
    num_trajectories_target = int(min_trajectories + (max_trajectories - min_trajectories) * (i / (len(E_list) - 1)))
    
    # Adiciona novas trajetórias se a lista atual for menor que o alvo
    trajectories_to_add = num_trajectories_target - len(selected_combinations)
    
    if trajectories_to_add > 0:
        # Pega os índices que ainda não foram usados
        available_indices = np.array(list(set(range(total_combinations)) - simulated_indices))
        
        if len(available_indices) > 0:
            # Seleciona aleatoriamente novos índices
            new_indices = np.random.choice(available_indices, min(trajectories_to_add, len(available_indices)), replace=False)
            
            # Adiciona as novas combinações à lista
            selected_combinations = np.vstack([selected_combinations, q2_p2_combinations[new_indices]])
            
            # Adiciona os novos índices ao conjunto de usados
            simulated_indices.update(new_indices)

    print(f"Iniciando simulações para E = {E_target} com {len(selected_combinations)} trajetórias...")

    trajectory_counter = 0
    for q2_0, p2_0 in selected_combinations:
        trajectory_counter += 1

        p1_sq = 2 * E_target - (p2_0**2 + q2_0**2) + (2/3) * q2_0**3

        if p1_sq >= 0:
            p1_calc = np.sqrt(p1_sq)
            initial_state = (0.0, q2_0, p1_calc, p2_0)

            # print(f"  > E = {E_target}: Trajetória {trajectory_counter}/{len(selected_combinations)}...")
            q2_points, p2_points = get_poincare_points(initial_state, E_target, dt, num_steps)

            all_q2_points.extend(q2_points)
            all_p2_points.extend(p2_points)

            num_valid_initials += 1

    if num_valid_initials > 0:
        plt.figure(figsize=(10, 10))
        plt.scatter(all_q2_points, all_p2_points, s=0.02, alpha=0.5)
        
        # Define os limites de visualização fixos
        plt.xlim(q2_min, q2_max)
        plt.ylim(p2_min, p2_max)

        plt.xlabel('$q_2$', fontsize=12)
        plt.ylabel('$p_2$', fontsize=12)
        plt.title(f'Seção de Poincaré (E = {E_target})', fontsize=14)
        plt.grid(True)
        
        filename = f'simulacao_{E_target:.4f}'.replace('.', '_')
        filepath = os.path.join(output_dir, filename)
        plt.savefig(filepath, bbox_inches='tight')
        
        print(f"Gráfico salvo como '{filepath}'")
        plt.close() # Fecha a figura para não sobrecarregar a memória
        # print(f"Concluído para E = {E_target}! Total de pontos: {len(all_q2_points)}\n")

    else:
        print(f"Nenhuma trajetória válida encontrada para E = {E_target}. Aumente o intervalo da grade ou a energia.\n")

Iniciando simulações para E = 0.005 com 350 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0050'
Iniciando simulações para E = 0.007 com 360 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0070'
Iniciando simulações para E = 0.0089 com 370 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0089'
Iniciando simulações para E = 0.01 com 381 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0100'
Iniciando simulações para E = 0.0117 com 391 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0117'
Iniciando simulações para E = 0.0133 com 402 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0133'
Iniciando simulações para E = 0.0148 com 412 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0148'
Iniciando simulações para E = 0.0167 com 423 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0167'
Iniciando simulações para E = 0.0183 com 433 trajetórias...
Gráfico salvo como 'imagens/simulacao_0_0183'
Iniciando simulações para E = 0.0188 com 444 traje